<a href="https://colab.research.google.com/github/darthapple/Imersao-IA-Alura-Google/blob/main/Gerar_Resumo_Noticias_Agregadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gerador de Resumo de Notícias

Projeto feito para a Imersão de IA Alura e Google.
Neste projeto utilizo o Gemni AI e Python para obter notícias de uma lista de feeds RSS e com a ajuda do Gemini agrupar noticias semelhantes e gerar um resumo delas, formando um documento a ser enviado por e-mail com o resumo diário de notícias.

## Instalar e importar os pacotes de de apoio

In [1]:
!pip install -U -q feedparser google-generativeai mistletoe

In [2]:
import feedparser
import google.generativeai as genai
import matplotlib.pyplot as plt
import mistletoe
import numpy as np
import pandas as pd
import pytz
import time

from datetime import date, datetime
from google.colab import userdata
from IPython.display import display, HTML
# from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine
from tqdm.auto import tqdm

### Configurar a API KEY do Gemini

In [3]:
genai.configure(api_key=userdata.get('GENAI_API_KEY'))

## Obter notícias e gerar um dataframe pandas com a lista
### Definir funções de apoio para baixar as notícias dos feeds e adicionar ao ```Data Frame``` de notícias

Transforma a ```string```de data em um objeto data e faz ajustes para que não tenha dados em português


In [4]:
def parse_date(data:str) -> date:
  # print("Data:", data)
  data = data.replace("Fev", "Feb")
  data = data.replace("Abr", "Apr")
  data = data.replace("Mai", "May")
  data = data.replace("Ago", "Aug")
  data = data.replace("Set", "Sep")
  data = data.replace("Out", "Oct")
  data = data.replace("Dez", "Dec")

  data = data.replace("Dom", "Sun")
  data = data.replace("Seg", "Mon")
  data = data.replace("Ter", "Tue")
  data = data.replace("Qua", "Wed")
  data = data.replace("Qui", "Thu")
  data = data.replace("Sex", "Fri")
  data = data.replace("Sáb", "Sat")

  return datetime.strptime(data, "%a, %d %b %Y %H:%M:%S %z")

Carrega as notícias diretamente do feed e filtra pela data base informada

In [5]:
def carregar_noticias(feed_link:str, data:date=datetime.now(pytz.timezone('America/Sao_Paulo')).date()) -> pd.DataFrame:
  noticias = []
  feed = feedparser.parse(feed_link)
  for entry in feed.entries:
      data_pub = parse_date(entry.published)

      if data_pub.date() >= data:
        noticia = {}
        noticia["link"] = entry.link
        noticia["title"] = entry.title
        noticia["text"] = entry.summary
        noticia["publihed"] = entry.published

        noticias.append(noticia)

  return pd.DataFrame(noticias)

### Carregar Notícias
Carrega as notícias de 3 fontes:
- G1: https://g1.globo.com/dynamo/tecnologia/rss2.xml
- UOL: https://rss.uol.com.br/feed/tecnologia.xml
- Wired: https://www.wired.com/feed/rss

Este último obtém notícias em inglês.


In [6]:
data_base = date(2024, 5, 10 )
df_noticias = carregar_noticias("https://g1.globo.com/dynamo/tecnologia/rss2.xml", data_base)
df_noticias = pd.concat([df_noticias, carregar_noticias("https://rss.uol.com.br/feed/tecnologia.xml", data_base)])
df_noticias = pd.concat([df_noticias, carregar_noticias("https://www.wired.com/feed/rss", data_base)])

In [7]:
print(f"Localizadas {df_noticias.shape[0]} notícias")

Localizadas 37 notícias


## Identificar e agrupar noticias similares

Gera um resumo de um texto utilizando o Gemini 1.0 Pró

In [8]:
def resumo_embed(texto:str) -> str:
  # Set up the model
  generation_config = {
    "temperature": 0.9,
    "top_p": 1,
    "top_k": 0,
    "max_output_tokens": 2048,
  }

  safety_settings = [
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
  ]

  model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                                generation_config=generation_config,
                                safety_settings=safety_settings)

  convo = model.start_chat(history=[
    {
      "role": "user",
      "parts": ["resumir o texto em até 3 paragrafos sem colocar **Parágrafo n:** onde n é o número do parágrafo:"]
    },
    {
      "role": "model",
      "parts": [texto]
    },
  ])

  convo.send_message(texto)
  return convo.last.text

Função para calcular o embedding da notícia. Se o texto for maior que 9000 caracteres usa o Gemini 1.5 Pró senão utiliza o 1.0 Pró.

In [9]:
def calcular_embed(title:str, text:str, model:str="models/embedding-001"):
  embeddings = []
  resumo = text
  if len(text) > 9000:
    resumo = resumo_embed(text)

  # print(title, len(resumo))
  try:
    embeddings = genai.embed_content(model=model,
                                  content=resumo,
                                  title=title,
                                  task_type="RETRIEVAL_DOCUMENT")["embedding"]
  except:
    pass

  return embeddings

def cosine_similarity(embedding1, embedding2):
  # Calcular a distância cosseno
  distance = cosine(embedding1, embedding2)
  # Retornar 1 - distância cosseno para obter similaridade
  return 1 - distance

Gerar o embbeding para todas as notícias

In [10]:
df_noticias["embedings"] = df_noticias.apply(lambda row: calcular_embed(row["title"], row["text"], "models/text-embedding-004"), axis=1)

#### Determinar a similaridade das notícias

Utiliza a a similaridade de coseno para calcular um valor entre 0 e 1, onde 0 não é similar e 1 é similar.

In [11]:
embeddings = df_noticias['embedings'].to_numpy()

n = embeddings.shape[0]  # Número de linhas no dataframe
similarity_matrix = np.zeros((n, n))

for i in range(n):
  for j in range(i + 1, n):
    similarity_matrix[i, j] = cosine_similarity(embeddings[i], embeddings[j])
    similarity_matrix[j, i] = similarity_matrix[i, j]  # Matriz simétrica

Com base no calculado anteriormente gera uma matriz com as notícias similares

In [12]:
def most_similar_embeddings(embeddings, similarity_matrix, threshold=0.75):
  """
  Encontra os embeddings mais similares para cada linha no dataframe.

  Args:
    embeddings: Array NumPy contendo os embeddings.
    similarity_matrix: Matriz NumPy de similaridade entre embeddings.

  Returns:
    Um dicionário onde cada chave é o índice da linha e o valor é um tupla contendo o índice do embedding mais similar e sua similaridade.
  """

  most_similar = {}
  for i in range(len(embeddings)):
    # Encontrar o índice do embedding mais similar (maior valor na coluna)
    most_similar_idx = np.argmax(similarity_matrix[i])
    # Obter a similaridade
    similarity = similarity_matrix[i, most_similar_idx]
    # Armazenar na lista
    if similarity > threshold:
      most_similar[i] = (most_similar_idx, similarity)

  return most_similar

# Exemplo de uso

Adiciona ao dataframe as notícias similares, evitando repetição

In [13]:
def add_relacao(idx, idx2):
  if idx2 not in df_noticias.iloc[idx]["relacionadas"]:
    df_noticias.iloc[idx]["relacionadas"].append(idx2)

df_noticias["relacionadas"] = df_noticias.apply(lambda row: [], axis=1)
similares = most_similar_embeddings(embeddings, similarity_matrix, 0.7)
for idx in similares:
  idx2, _ = similares[idx]
  add_relacao(idx, idx2)
  add_relacao(idx2, idx)

## Gerar Resumo das notícias

Utilizando o Gemnini 1.5 Pro gera a lista de noticias com seu resumo.

In [14]:
# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

system_instruction = "Como um reporter especializado em tecnologia, porém, focado em escrever para um grupo de nerds, sem muita formalidade, mas, com uma escrita profissional sem uso excessivo de gírias"
# system_instruction = "Como um reporter especializado em tecnologia, porém, focado em escrever para um grupo de nerds, sem muita formalidade, mas, com uma escrita profissional"


def resumir(texto:str) -> str:
  modelo = "gemini-1.5-pro-latest"
  # modelo = "gemini-1.0-pro"
  model = genai.GenerativeModel(model_name=modelo,
                                generation_config=generation_config,
                                system_instruction=system_instruction,
                                safety_settings=safety_settings)
  convo = model.start_chat(history=[
    {
      "role": "user",
      "parts": ["Informação:\nAbaixo temos uma lista de notícias sobre um mesmo tema e separadas por ----- e elas podem ser em várias línguas diferentes.\n\nObjetivo:\nResumir em dois parágrafos o que as noticias estão falando e gerar um título para elas na primeira linha."]
    },
  ])

  pausa = 30
  resumo = ""
  for retry in range(5):
    try:
      convo.send_message(noticia.text)

      resumo = convo.last.text
      break
    except:
      time.sleep(pausa)
      pausa += 10

  return resumo


In [15]:
resumo = ""
resumo_md = ""
for idx, noticia in tqdm(df_noticias.iterrows(), total=df_noticias.shape[0], desc="Gerando Resumos"):
  resenha = resumir(noticia)
  resenha += f"\n\nFonte: [{noticia.link}]({noticia.link})"

  if len(noticia.relacionadas) > 0:
    resenha += "\n\nRelacionadas:"
    for idx in noticia.relacionadas:
      resenha += f"\n\n   [{noticia.title}]({noticia.link})"
  resumo += mistletoe.markdown(resenha)
  resumo_md += resenha
  time.sleep(15)

display(HTML(resumo))

Gerando Resumos:   0%|          | 0/37 [00:00<?, ?it/s]